# Data API
### Emoji Sentiment ( MySQL )

In [1]:
import os
import re
import urllib.request

source = urllib.request.urlopen('https://unicode.org/emoji/charts/full-emoji-list.html')

pattern1 = r'<td class=\'code\'><a href=\'\#([A-Fa-f0-9_]+)\' name=\'([A-Fa-f0-9_]+)\'>.+</a></td>'
pattern2 = r'<td class=\'chars\'>(.+)</td>'
pattern3 = r'<td class=\'name\'>(.+)</td>'

code = []
char = []
desc = []

for line in source.readlines():
    line = line.decode('utf-8').strip()
    if line[:10] == '<td class=':
        if line[11:15]  == 'code':
            match = re.match(pattern1, line)
            if match is not None:
                    code.append(match.group(1).lower())
        elif line[11:15]  == 'char':
            match = re.match(pattern2, line)
            if match is not None:
                    char.append(match.group(1).lower())
        elif line[11:15]  == 'name':
            match = re.match(pattern3, line)
            if match is not None:
                    desc.append(match.group(1).lower())

print(len(code), len(char), len(desc))

2623 2623 2623


In [2]:
len_code = 0
len_desc = 0
num_code = 0
len_char = 0

for i in range(len(code)):
    code[i] = code[i].split('_')
    num_code = max(num_code, len(code[i]))
    code[i] = ' '.join([str(ord(chr(int(c, base=16)))) for c in code[i]])
    len_code = max(len_code, len(code[i]))
    len_char = max(len_char, len(char[i]))
    len_desc = max(len_desc, len(desc[i]))

print(num_code, len_char, len_code, len_desc)

8 8 48 53


Create database with sentiment scores initialzed to 0:

In [3]:
insert = '''
        INSERT INTO emoji(code, chars, composite, description)
        VALUES('{}','{}',{},'{}');
        '''

with open('emoji.sql','w') as output:
    output.write('''
        DROP TABLE IF EXISTS emoji;
        CREATE TABLE emoji(
            code VARCHAR({}) NOT NULL DEFAULT '',
            chars VARCHAR({}) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci NOT NULL DEFAULT '',
            composite TINYINT(1) NOT NULL DEFAULT 0,
            description VARCHAR({}) NOT NULL DEFAULT '',
            PRIMARY KEY (code)
        ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
        '''.format(len_code, num_code, len_desc))

    for i in range(len(code)):
        description = ''.join(filter(lambda c: ord(c) < 128, desc[i]))
        output.write(insert.format(code[i], ''.join(char[i]), len(code[i].split()), description))

Import data to the table:
<pre>
mysql -u root -p$MYSQL_ROOT_PASS $DATABASE &lt; emoji.sql
</pre>
Attention: client-server **configuration** might need to be revised ( /etc/mysql/my.cnf )
<pre>
[client]
default-character-set = utf8mb4
[mysql]
default-character-set = utf8mb4
[mysqld]
character-set-client-handshake = FALSE
character-set-server = utf8mb4
collation-server = utf8mb4_unicode_ci
</pre>
Now let's run some simple positive and negative queries:

In [4]:
import os
import MySQLdb as sql
import pandas as pd

con = sql.connect(host = 'localhost', database = os.environ['DATABASE'],
                  user = os.environ['DATAUSER'], password = os.environ['MYSQL_PASS'])

data = pd.read_sql("""
SELECT a.description, b.positive, b.negative, b.total, b.sentiment
FROM emoji a JOIN emo_sent b USING(code) WHERE total > 0
""", con)
data = pd.DataFrame(data)

data[data['sentiment'] > 0].sort_values(by=['total','sentiment'], ascending=[0,1]).head(10)

,description,positive,negative,total,sentiment
239,face with tears of joy,1194,208,1402,0.703281
7,red heart,947,6,953,0.987408
250,smiling face with heart-eyes,504,3,507,0.988166
282,loudly crying face,365,76,441,0.655329
220,fire,422,3,425,0.985882
180,blue heart,230,1,231,0.991342
336,rolling on the floor laughing,187,23,210,0.780952
308,folded hands,196,5,201,0.950249
176,two hearts,194,3,197,0.969543
137,clapping hands,185,11,196,0.887755


In [5]:
data[data['sentiment'] < 0].sort_values(by=['total','sentiment'], ascending=[0,1]).head(10)

,description,positive,negative,total,sentiment
191,pile of poo,4,115,119,-0.932773
338,person facepalming,12,40,52,-0.538462
275,frowning face with open mouth,0,51,51,-1.000000
255,unamused face,5,28,33,-0.696970
0,heavy check mark,6,8,14,-0.142857
136,thumbs down,2,8,10,-0.600000
233,middle finger,2,7,9,-0.555556
254,expressionless face,3,5,8,-0.250000
25,new moon face,1,5,6,-0.666667
272,persevering face,1,5,6,-0.666667


In [ ]:
# test run
!uwsgi --http :9000 --wsgi-file app.py --callable app

And here it is [online](/api/twitter/). In our emo-sent job ([python](../python)) we used runtime as proxy for the number of messages, however, as it appears, the positive content is strongly dominating with the ratio about 10 to 1.

In [6]:
import pandas as pd
from viz import local
viz = local.VizNotebook()

In [7]:
df = pd.read_json('http://www.arcta.me/api/twitter/emo/ratio/history')
df.head()

,0,1
0,2017-12,0.102283
1,2017-11,0.112300


In [8]:
df = pd.read_json('http://www.arcta.me/api/twitter/geo/sent/history/7')
df.head()

,0,1,2
0,Algeria,0.075532,2017-12-05
1,Argentina,0.033679,2017-12-05
2,Argentina,0.066029,2017-12-04
3,Argentina,0.101902,2017-12-03
4,Argentina,0.118823,2017-12-02


In [9]:
df.columns = ['Country','Score','Date']
df.Date = pd.to_datetime(df.Date)
df.head()

,Country,Score,Date
0,Algeria,0.075532,2017-12-05
1,Argentina,0.033679,2017-12-05
2,Argentina,0.066029,2017-12-04
3,Argentina,0.101902,2017-12-03
4,Argentina,0.118823,2017-12-02


In [10]:
geosent = df.groupby(['Country'], axis=0).mean().sort_values('Score', ascending = False)
geosent.head(10)

,Score
Country,
Swaziland,0.255854
Belarus,0.179602
Saudi Arabia,0.163929
Uruguay,0.162834
Kuwait,0.162463
Israel,0.161378
Viet Nam,0.161354
Iraq,0.158915
Italy,0.158288


In [11]:
geosent.tail(10)

,Score
Country,
Tanzania,0.019528
Haiti,0.016946
Panama,0.015376
Serbia,0.005209
Trinidad and Tobago,-0.004256
Dominican Republic,-0.007198
Mozambique,-0.009137
Romania,-0.045844
Zambia,-0.061319
